----


### Aşağıda birkaç farklı senaryo için net PyTorch örnekleri çözelim.

* Örnek 1: Sadece kanal sayısını değiştiren basit pointwise conv

* Örnek 2: Depthwise + Pointwise bloğu (mobil mimari stili)

* Örnek 3: Feature fusion için pointwise kullanımı

* Örnek 4: Linear yerine “global average pooling + 1×1 conv” kullanımı
----

## Örnek 1 – Basit Pointwise: C_in → C_out

* Klasik kullanım: kanal sayısını değiştirmek ve kanalları karıştırmak.

In [1]:
import torch
import torch.nn as nn

N,C_in ,H,W = 4,32,32,32
x = torch.randn(N,C_in,H,W)

pointwise = nn.Conv2d(
    in_channels=32,
    out_channels=64,
    kernel_size=1,
    stride=1,
    padding=0,
    bias=False)

y = pointwise(x)

print("Girdi şekli :", x.shape)   # (4, 32, 32, 32)
print("Çıktı şekli :", y.shape)   # (4, 64, 32, 32)

print("Parametre sayısı:", sum(p.numel() for p in pointwise.parameters()))

Girdi şekli : torch.Size([4, 32, 32, 32])
Çıktı şekli : torch.Size([4, 64, 32, 32])
Parametre sayısı: 2048


Burada kritik noktalar:

* H ve W değişmiyor

* Sadece kanal sayısı değişiyor → 32 → 64

* Her (h, w) lokasyonunda aslında 32 → 64 boyutlu bir FC layer çalışıyor gibi düşünebilirsin.

## Örnek 2 – Depthwise + Pointwise Bloğu (Depthwise Separable)

Bu blok:

* Uzamsal işlemi depthwise ile yapar

* Kanal karışımını ve C_in → C_out dönüşümünü pointwise ile yapar

In [3]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, c_in, c_out, kernel_size=3, stride=1, padding=1):
        super().__init__()
        # 1) Depthwise: her kanal kendi filtresiyle (uzamsal)
        self.depthwise = nn.Conv2d(
            in_channels=c_in,
            out_channels=c_in,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            groups=c_in,    # depthwise
            bias=False
        )
        # 2) Pointwise: kanal karışımı + kanal sayısı değişimi
        self.pointwise = nn.Conv2d(
            in_channels=c_in,
            out_channels=c_out,
            kernel_size=1,
            bias=False
        )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x


# Küçük test:
c_in, c_out = 32, 64
x = torch.randn(1, c_in, 32, 32)

std_conv = nn.Conv2d(c_in, c_out, kernel_size=3, padding=1, bias=False)
ds_conv  = DepthwiseSeparableConv(c_in, c_out, kernel_size=3, padding=1)

y_std = std_conv(x)
y_ds  = ds_conv(x)

def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

print("Standart 3x3 Conv parametre sayısı:", count_params(std_conv))
print("Depthwise+Pointwise parametre sayısı:", count_params(ds_conv))
print("Standart Conv çıktı şekli        :", y_std.shape)
print("Depthwise+Pointwise çıktı şekli  :", y_ds.shape)


Standart 3x3 Conv parametre sayısı: 18432
Depthwise+Pointwise parametre sayısı: 2336
Standart Conv çıktı şekli        : torch.Size([1, 64, 32, 32])
Depthwise+Pointwise çıktı şekli  : torch.Size([1, 64, 32, 32])


Burada pointwise:

* Depthwise’ın ürettiği (N, C_in, H, W) feature’ları alıp

* Hem karıştırıyor hem de C_out kanal sayısına çeviriyor.

## Örnek 3 – Feature Fusion için Pointwise Kullanımı

* Senaryon: iki farklı branch’ten gelen feature map’leri birleştiriyorsun.
* Mesela skip connection veya multi-scale feature fusion.

In [4]:
class FeatureFusionWithPointwise(nn.Module):
    def __init__(self, c_in1 , c_in2 , c_out):
        super().__init__()

        self.fuse = nn.Conv2d(
            in_channels=c_in1+c_in2,
            out_channels=c_out,
            kernel_size=1,
            bias=True)
    
    def forward(self,x1,x2):
        x = torch.cat([x1,x2] , dim = 1)
        x = self.fuse(x)
        return x 
    
# Test:
x1 = torch.randn(1, 32, 16, 16)
x2 = torch.randn(1, 64, 16, 16)

fusion = FeatureFusionWithPointwise(c_in1=32, c_in2=64, c_out=48)
y = fusion(x1, x2)

print("Concat sonrası şekil:", torch.cat([x1, x2], dim=1).shape)  # (1, 96, 16, 16)
print("Fusion çıkışı şekil:", y.shape)  


Concat sonrası şekil: torch.Size([1, 96, 16, 16])
Fusion çıkışı şekil: torch.Size([1, 48, 16, 16])


Burada pointwise:

* Concat ile oluşan 96 kanallı feature’ı alıp

* Hem karıştırıyor hem 48 kanala indiriyor.

## Örnek 4 – Linear Yerine GAP + 1×1 Conv

* Özellikle segmentation / detection backbone’larında şu pattern çok yaygın:

**Global Average Pool + 1×1 Conv = Fully Connected layer’in konvolüsyonel karşılığı**

In [5]:
class GAPPlusPointwiseHead(nn.Module):
    def __init__(self, c_in, num_classes):
        super().__init__()
        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # (N, C_in, H, W) -> (N, C_in, 1, 1)
        self.classifier = nn.Conv2d(
            in_channels=c_in,
            out_channels=num_classes,
            kernel_size=1,
            bias=True
        )

    def forward(self, x):
        x = self.gap(x)            # (N, C_in, 1, 1)
        x = self.classifier(x)     # (N, num_classes, 1, 1)
        x = x.view(x.size(0), -1)  # (N, num_classes)
        return x


# Test:
x = torch.randn(4, 128, 7, 7)          # ResNet'ten gelen son feature gibi
head = GAPPlusPointwiseHead(128, 10)   # 10 sınıflı output
y = head(x)
print("Çıktı şekli:", y.shape)         # (4, 10)


Çıktı şekli: torch.Size([4, 10])


Burada:

* FC yerine 1×1 conv kullanıyorsun

* Tamamen konvolüsyonel kalmak istiyorsan (fully-convolutional network) bu çok iş görüyor.